In [1]:
#%%capture
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption = 9
centralities = ["0%-93%"]
Opt_names = ["std", "under","over"]

In [3]:
N_centr = 6
N_options = 6
N_pt = 1
colors0=[1,2,root.kGreen+2,4,root.kMagenta,root.kOrange+4,root.kGray,root.kGray,root.kBlack,root.kYellow,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
colors=[2,4,4,root.kGreen+2,root.kOrange+4,root.kMagenta,root.kGray,root.kGray,root.kBlack,root.kCyan,root.kOrange,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="output/DCA_templates/"
file_names=["newmass0.root","newmass2.root","newmass3.root"]
hist_names = [f"inv_mass_ee_DCA_V{iOption}_FG120_proj_mass00proj_reb",f"inv_mass_ee_DCA_V{iOption}_FG120_proj_syst_mass00proj_syst_reb"]
N_type = len(hist_names)

In [4]:
hists_read = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hists_emc_read0.append(infile.Get(hist_names[itype]))
        try: hists_emc_read0[-1].SetDirectory(root.nullptr); hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"_{iFile}")
        except: 
            print(f"Error setting directory for {hist_names[itype]}{0} in file {file_names[iFile]}")
            break
        hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"{iFile}")
    hists_read.append(hists_emc_read0)

    infile.Close()


In [5]:
c0 = root.TCanvas(f"c0",f"c0",900,900)
legends =[]
hist0_stat = hists_read[0][0].Clone(); hist0_syst = hists_read[0][1].Clone()
hist1_stat = hists_read[1][0].Clone(); hist1_syst = hists_read[1][1].Clone()
hist2_stat = hists_read[2][0].Clone(); hist2_syst = hists_read[2][1].Clone()


Format_Hist_total(hist0_stat,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
hist0_stat.SetMarkerColorAlpha(colors0[0],(10-1)/10.); hist0_stat.SetLineColorAlpha  (colors0[0],(10-1)/10.)
hist0_syst.SetFillColorAlpha(colors0[0],0.4)

hist1_stat.SetMarkerColorAlpha(colors0[1],(10-1)/10.); hist1_stat.SetLineColorAlpha  (colors0[1],(10-1)/10.)
hist1_syst.SetFillColorAlpha(colors0[1],0.4)

hist2_stat.SetMarkerColorAlpha(colors0[2],(10-1)/10.); hist2_stat.SetLineColorAlpha  (colors0[2],(10-1)/10.)
hist2_syst.SetFillColorAlpha(colors0[2],0.4)

hist0_stat.Draw(""); hist0_syst.Draw("same e2")
hist2_stat.Draw("same"); hist2_syst.Draw("same e2")
hist1_stat.Draw("same"); hist1_syst.Draw("same e2")

root.gPad.SetLogy()
c0.Draw()


In [6]:
c1 = root.TCanvas(f"c1",f"c1",900,900)
legends =[]
hist0_new_stat = hist0_stat.Rebin(1, hist0_stat.GetName()+"hist0_stat_rebinned"); hist0_new_syst = hist0_syst.Rebin(1, hist0_syst.GetName()+"hist0_syst_rebinned")
hist1_new_stat = hist1_stat.Rebin(1, hist1_stat.GetName()+"hist1_stat_rebinned"); hist1_new_syst = hist1_syst.Rebin(1, hist1_syst.GetName()+"hist1_syst_rebinned")
hist2_new_stat = hist2_stat.Rebin(1, hist2_stat.GetName()+"hist2_stat_rebinned"); hist2_new_syst = hist2_syst.Rebin(1, hist2_syst.GetName()+"hist2_syst_rebinned")  



for ibin in range(1,hist0_new_stat.GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
    val0 = hist0_new_stat.GetBinContent(ibin)
    val1 = hist1_new_stat.GetBinContent(ibin)
    val2 = hist2_new_stat.GetBinContent(ibin)
    if val0>0:
        hist1_new_stat.SetBinContent(ibin, val1/val0)
        hist2_new_stat.SetBinContent(ibin, val2/val0)
        err1 = hist1_new_stat.GetBinError(ibin)
        err2 = hist2_new_stat.GetBinError(ibin)
        hist1_new_stat.SetBinError(ibin, err1/val0)
        hist2_new_stat.SetBinError(ibin, err2/val0)
        hist0_new_syst.SetBinContent(ibin, 1)
        hist0_new_syst.SetBinError(ibin, hist0_new_syst.GetBinError(ibin)/val0)
    else:
        hist1_new_stat.SetBinContent(ibin, 0)
        hist2_new_stat.SetBinContent(ibin, 0)
        hist1_new_stat.SetBinError(ibin, 0)
        hist2_new_stat.SetBinError(ibin, 0)

Format_Hist_total(hist1_new_stat,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
hist0_new_stat.SetMarkerColorAlpha(colors0[0],(10-1)/10.); hist0_new_stat.SetLineColorAlpha  (colors0[0],(10-1)/10.)
hist0_new_syst.SetFillColorAlpha(colors0[0],0.3)

hist1_new_stat.SetMarkerColorAlpha(colors0[1],(10-1)/10.); hist1_new_stat.SetLineColorAlpha  (colors0[1],(10-1)/10.)
hist1_new_syst.SetFillColorAlpha(colors0[1],0.2)

hist2_new_stat.SetMarkerColorAlpha(colors0[2],(10-1)/10.); hist2_new_stat.SetLineColorAlpha  (colors0[2],(10-1)/10.)
hist2_new_syst.SetFillColorAlpha(colors0[2],0.2)

hist1_new_syst.Divide(hist0_new_syst); hist2_new_syst.Divide(hist0_new_syst)
#root.gPad.SetLogy(1)
hist1_new_stat.SetMaximum(2); hist1_new_stat.SetMinimum(0.1)
hist1_new_stat.Draw(""); #hist1_new_syst.Draw("same e2")
hist2_new_stat.Draw("same"); #hist2_new_syst.Draw("same e2")
hist0_new_syst.Draw("same e2")
#drawing dashed line at y=1
line = root.TLine(hist1_new_stat.GetXaxis().GetXmin(), 1, hist1_new_stat.GetXaxis().GetXmax(), 1)
line.SetLineStyle(2); line.SetLineWidth(4); line.SetLineColor(root.kBlack)
line.Draw("same")

c1.Draw()


In [7]:
import ROOT as root
import numpy as np
import math

# --------------------------------------------------------------------
# Helper: µ_B(√s_NN) parameterization (Andronic et al.)
# µ_B(√s) [GeV] = a / (1 + b * √s),  a≈1.308 GeV, b≈0.273 GeV^-1
# We convert to MeV here.
# --------------------------------------------------------------------
def muB_mev(sNN_GeV):
    a = 1.308  # GeV
    b = 0.273  # 1/GeV
    return 1000.0 * a / (1.0 + b * sNN_GeV)

# For now, put placeholders so the cell runs without your fit:
T_eff_yours      = np.array([262,271,260])  # placeholder values # bg 500. bg 300, 500, 300, bg mass
T_eff_yours_err  = np.array([30,34,28])
mu_B_yours       = np.array([muB_mev(200.0), muB_mev(220.0), muB_mev(180.0)])  # placeholder values
T_eff_avg = np.mean(T_eff_yours)
T_eff_err_avg = max(T_eff_yours_err)
std_dev = np.std(T_eff_yours)+25

# √s_NN values used in Fig. 4
s_HADES  = 2.42
s_NA60   = 17.3
s_STAR27 = 27.0
s_STAR54 = 54.4
s_BES19  = 19.6
s_BES39  = 39.0
s_BES62  = 62.4
s_BES200 = 200.0

# --------------------------------------------------------------------
# µ_B positions in MeV (at chemical freeze-out)
# --------------------------------------------------------------------
mu_B_hades   = muB_mev(s_HADES)
mu_B_na60    = muB_mev(s_NA60)
mu_B_star27  = muB_mev(s_STAR27)
mu_B_star54  = muB_mev(s_STAR54)

# For STAR BES-I LMR points – approximate:
mu_B_star_bes = np.array([
    muB_mev(s_BES200),  # ≈ 24 MeV
    muB_mev(s_BES62),   # ≈ 73 MeV
    muB_mev(s_BES39),   # ≈112 MeV
    muB_mev(s_BES19),   # ≈206 MeV
])

# --------------------------------------------------------------------
# Temperatures and errors from the text (LMR & IMR)
#   LMR STAR 27 GeV:  165 ±20(stat) ±21(syst) MeV
#   LMR STAR 54.4 GeV:178 ±15(stat) ±13(syst) MeV
#   IMR STAR 27 GeV:  274 ±65(stat) ±10(syst) MeV
#   IMR STAR 54.4 GeV:287 ±70(stat) ±34(syst) MeV
#   LMR NA60:         172 ± 6 MeV
#   IMR NA60:         245 ±17 MeV
#   LMR HADES:        71.8 ±2.1 MeV
# We combine stat⊕syst in quadrature for a single symmetric error.
# --------------------------------------------------------------------
T_star_lmr   = np.array([178.0, 165.0])  # (54.4, 27) – we'll keep µ_B in same order below
T_star_lmr_err = np.array([
    math.sqrt(15.0**2 + 13.0**2),  # 54.4
    math.sqrt(20.0**2 + 21.0**2),  # 27
])

T_star_imr   = np.array([287.0, 274.0,])  # (54.4, 27)
T_star_imr_err = np.array([
    math.sqrt(70.0**2 + 34.0**2),  # 54.4
    math.sqrt(65.0**2 + 10.0**2),  # 27
])

T_na60_lmr      = np.array([172.0])
T_na60_lmr_err  = np.array([6.0])

T_na60_imr      = np.array([245.0])
T_na60_imr_err  = np.array([17.0])

T_hades_lmr     = np.array([71.8])
T_hades_lmr_err = np.array([2.1])

# --------------------------------------------------------------------
# APPROXIMATE STAR BES-I LMR points (from Fig. 4 *by eye*):
# they all sit close to the LQCD band around 155–165 MeV.
# Adjust later if/when you pull exact HEPData numbers.
# Order corresponds to mu_B_star_bes defined above.
# --------------------------------------------------------------------
T_star_bes_lmr = np.array([160.0, 160.0, 162.0, 158.0])
T_star_bes_lmr_err = np.array([40.0, 40.0, 45.0, 45.0])  # big errors as in the figure

# µ_B arrays for STAR LMR/IMR (54.4, 27) in same order as the T arrays
mu_B_star_pair = np.array([mu_B_star54, mu_B_star27])

# --------------------------------------------------------------------
# Your measurement (from fit_function: M^(3/2) * exp(-M/T))
# fit_function must already exist (e.g. from the IMR fit in your macro).
# --------------------------------------------------------------------
# Example:
# T_eff_yours = fit_function.GetParameter(1) * 1000.0  # GeV -> MeV
# T_eff_yours_err = fit_function.GetParError(1) * 1000.0
# mu_B_yours = muB_mev(200.0)  # if your point is Au+Au 200 GeV

print(f"Your T_eff measurement (placeholder): {T_eff_avg:.1f} ± {T_eff_err_avg:.1f} ± {std_dev:.1f} MeV")
print(f"STAR IMR: 54.4 GeV → {T_star_imr[0]:.1f} ± {T_star_imr_err[0]:.1f} MeV at µ_B≈{mu_B_star_pair[0]:.0f} MeV")
print(f"          27  GeV → {T_star_imr[1]:.1f} ± {T_star_imr_err[1]:.1f} MeV at µ_B≈{mu_B_star_pair[1]:.0f} MeV")

# --------------------------------------------------------------------
# Draw Fig. 4-style plot in ROOT
# --------------------------------------------------------------------
c_temp = root.TCanvas("c_Teff_comparison", "Temperature vs #mu_{B}", 1000, 800)
root.gPad.SetLogx()
#root.gPad.SetGridx()
#root.gPad.SetGridy()

frame = root.TH2F("frame", ";#mu_{B} [MeV];T [MeV]",
                  100, 5.0, 1000.0,
                  100, 0.0, 500.0)
root.gPad.SetLeftMargin(0.12);root.gPad.SetBottomMargin(0.15);root.gPad.SetRightMargin(0.03);root.gPad.SetTopMargin(0.03)
frame.GetXaxis().SetTitleSize(0.06);frame.GetYaxis().SetTitleSize(0.06);frame.GetXaxis().SetLabelSize(0.05);frame.GetYaxis().SetLabelSize(0.05);frame.GetYaxis().SetNdivisions(505)
frame.GetXaxis().CenterTitle(); frame.GetYaxis().CenterTitle()
frame.GetYaxis().SetTitleOffset(0.9)#;frame.GetYaxis().SetTitleOffset(1.2)
frame.Draw()

# --- LQCD Tc band (approx 156 ± 5 MeV) ---
box_lqcd = root.TBox(5.0, 150.0, 1000.0, 163.0)
box_lqcd.SetFillColorAlpha(root.kOrange + 1, 0.35)
box_lqcd.SetLineColor(root.kOrange + 1)
box_lqcd.Draw("same")

# --- STAR IMR (red stars) ---
gr_star_imr = root.TGraphErrors(len(mu_B_star_pair))
for i in range(len(mu_B_star_pair)):
    gr_star_imr.SetPoint(i, mu_B_star_pair[i], T_star_imr[i])
    gr_star_imr.SetPointError(i, 0.0, T_star_imr_err[i])
gr_star_imr.SetMarkerStyle(29)
gr_star_imr.SetMarkerColor(root.kRed)
gr_star_imr.SetMarkerSize(2.5)
gr_star_imr.SetLineColor(root.kRed)
gr_star_imr.SetLineWidth(2)
gr_star_imr.Draw("P same")

# --- STAR LMR (blue stars) ---
gr_star_lmr = root.TGraphErrors(len(mu_B_star_pair))
for i in range(len(mu_B_star_pair)):
    gr_star_lmr.SetPoint(i, mu_B_star_pair[i], T_star_lmr[i])
    gr_star_lmr.SetPointError(i, 0.0, T_star_lmr_err[i])
gr_star_lmr.SetMarkerStyle(29)
gr_star_lmr.SetMarkerColor(root.kBlue)
gr_star_lmr.SetMarkerSize(2.5)
gr_star_lmr.SetLineColor(root.kBlue)
gr_star_lmr.SetLineWidth(2)
gr_star_lmr.Draw("P same")

# --- NA60 IMR (open red diamond) ---
gr_na60_imr = root.TGraphErrors(1)
gr_na60_imr.SetPoint(0, mu_B_na60, T_na60_imr[0])
gr_na60_imr.SetPointError(0, 0.0, T_na60_imr_err[0])
gr_na60_imr.SetMarkerStyle(27)
gr_na60_imr.SetMarkerColor(root.kRed)
gr_na60_imr.SetMarkerSize(2.0)
gr_na60_imr.SetLineColor(root.kRed)
gr_na60_imr.Draw("P same")

# --- NA60 LMR (open blue diamond) ---
gr_na60_lmr = root.TGraphErrors(1)
gr_na60_lmr.SetPoint(0, mu_B_na60, T_na60_lmr[0])
gr_na60_lmr.SetPointError(0, 0.0, T_na60_lmr_err[0])
gr_na60_lmr.SetMarkerStyle(27)
gr_na60_lmr.SetMarkerColor(root.kBlue)
gr_na60_lmr.SetMarkerSize(2.0)
gr_na60_lmr.SetLineColor(root.kBlue)
gr_na60_lmr.Draw("P same")

# --- HADES LMR (inverted blue triangle) ---
gr_hades = root.TGraphErrors(1)
gr_hades.SetPoint(0, mu_B_hades, T_hades_lmr[0])
gr_hades.SetPointError(0, 0.0, T_hades_lmr_err[0])
gr_hades.SetMarkerStyle(23)
gr_hades.SetMarkerColor(root.kBlue)
gr_hades.SetMarkerSize(2.0)
gr_hades.SetLineColor(root.kBlue)
gr_hades.Draw("P same")

# --- STAR BES-I LMR (magenta squares, approximate) ---
gr_star_bes_lmr = root.TGraphErrors(len(mu_B_star_bes))
for i in range(len(mu_B_star_bes)):
    gr_star_bes_lmr.SetPoint(i, mu_B_star_bes[i], T_star_bes_lmr[i])
    gr_star_bes_lmr.SetPointError(i, 0.0, T_star_bes_lmr_err[i])
gr_star_bes_lmr.SetMarkerStyle(21)
gr_star_bes_lmr.SetMarkerColor(root.kMagenta + 1)
gr_star_bes_lmr.SetMarkerSize(1.8)
gr_star_bes_lmr.SetLineColor(root.kMagenta + 1)
gr_star_bes_lmr.Draw("P same")

# --- Your IMR point (green star) ---
gr_yours = root.TGraphErrors(len(mu_B_yours))
for i in range(len(mu_B_yours)):
    gr_yours.SetPoint(i, mu_B_yours[i], T_eff_yours[i])
    gr_yours.SetPointError(i, 0.0, T_eff_yours_err[i])
gr_yours.SetMarkerStyle(33)
gr_yours.SetMarkerColor(root.kGreen + 2)
gr_yours.SetMarkerSize(3.0)
gr_yours.SetLineColor(root.kGreen + 2)
gr_yours.SetLineWidth(3)
gr_yours.Draw("P same")

gr_my_average = root.TGraphErrors(1)
gr_my_average.SetPoint(0, np.mean(mu_B_yours), T_eff_avg)
gr_my_average.SetPointError(0, 0.0, T_eff_err_avg)
gr_my_average.SetMarkerStyle(20)
gr_my_average.SetMarkerColorAlpha(root.kGreen + 4, 0.7)
gr_my_average.SetMarkerSize(2.0)
gr_my_average.SetLineColorAlpha(root.kGreen + 4, 0.5)
gr_my_average_std = root.TGraphErrors(1)
gr_my_average_std.SetFillColorAlpha(root.kGreen + 4, 0.35)
gr_my_average.SetLineWidth(3)
#gr_my_average.Draw("P same")
gr_my_average_std = root.TGraphErrors(1)
gr_my_average_std.SetPoint(0, np.mean(mu_B_yours), T_eff_avg)
gr_my_average_std.SetPointError(0, 2, std_dev)
gr_my_average_std.SetMarkerStyle(20)
gr_my_average_std.SetMarkerSize(0.0)
gr_my_average_std.SetFillColorAlpha(root.kGreen + 4, 0.35)
gr_my_average_std.SetLineWidth(3)
#gr_my_average_std.Draw("p2 same")

# --------------------------------------------------------------------
# Legend (you probably already have Format_Legend; if not, just set basics)
# --------------------------------------------------------------------
legend_temp = root.TLegend(0.12, 0.65, 0.57, 0.97)
legend_temp.SetBorderSize(0)
legend_temp.SetFillStyle(0)
legend_temp.SetTextSize(0.035)
legend_temp.AddEntry(gr_star_imr,      "IMR STAR",             "p")
legend_temp.AddEntry(gr_star_lmr,      "LMR STAR",             "p")
legend_temp.AddEntry(gr_star_bes_lmr,  "LMR STAR BES-I (approx.)", "p")
legend_temp.AddEntry(gr_na60_imr,      "IMR NA60 (In+In)",     "p")
legend_temp.AddEntry(gr_na60_lmr,      "LMR NA60 (In+In)",     "p")
legend_temp.AddEntry(gr_hades,         "LMR HADES",            "p")
legend_temp.AddEntry(box_lqcd,         "T_{C} LQCD",           "f")
legend_temp.AddEntry(gr_my_average,
                     f"Average {T_eff_avg:.0f} #pm {T_eff_err_avg:.0f} #pm {std_dev:.0f} MeV",
                     "pef")
legend_temp.Draw()

text = root.TLatex()
text.SetNDC()
text.SetTextSize(0.04)
text.DrawLatex(0.65, 0.85, "PHENIX Au+Au")
text.DrawLatex(0.65, 0.80, "Thermal dielectrons")
#root.gStyle.SetOptStat(0)
c_temp.Draw()
#c_temp.SaveAs("output/Teff_vs_muB.pdf")


Your T_eff measurement (placeholder): 264.3 ± 34.0 ± 29.8 MeV
STAR IMR: 54.4 GeV → 287.0 ± 77.8 MeV at µ_B≈83 MeV
          27  GeV → 274.0 ± 65.8 MeV at µ_B≈156 MeV
